![Los Angeles skyline](la_skyline.jpg)

Los Angeles, California 😎. The City of Angels. Tinseltown. The Entertainment Capital of the World! 

Known for its warm weather, palm trees, sprawling coastline, and Hollywood, along with producing some of the most iconic films and songs. However, as with any highly populated city, it isn't always glamorous and there can be a large volume of crime. That's where you can help!

You have been asked to support the Los Angeles Police Department (LAPD) by analyzing crime data to identify patterns in criminal behavior. They plan to use your insights to allocate resources effectively to tackle various crimes in different areas.

## The Data

They have provided you with a single dataset to use. A summary and preview are provided below.

It is a modified version of the original data, which is publicly available from Los Angeles Open Data.

# crimes.csv

| Column     | Description              |
|------------|--------------------------|
| `'DR_NO'` | Division of Records Number: Official file number made up of a 2-digit year, area ID, and 5 digits. |
| `'Date Rptd'` | Date reported - MM/DD/YYYY. |
| `'DATE OCC'` | Date of occurrence - MM/DD/YYYY. |
| `'TIME OCC'` | In 24-hour military time. |
| `'AREA NAME'` | The 21 Geographic Areas or Patrol Divisions are also given a name designation that references a landmark or the surrounding community that it is responsible for. For example, the 77th Street Division is located at the intersection of South Broadway and 77th Street, serving neighborhoods in South Los Angeles. |
| `'Crm Cd Desc'` | Indicates the crime committed. |
| `'Vict Age'` | Victim's age in years. |
| `'Vict Sex'` | Victim's sex: `F`: Female, `M`: Male, `X`: Unknown. |
| `'Vict Descent'` | Victim's descent:<ul><li>`A` - Other Asian</li><li>`B` - Black</li><li>`C` - Chinese</li><li>`D` - Cambodian</li><li>`F` - Filipino</li><li>`G` - Guamanian</li><li>`H` - Hispanic/Latin/Mexican</li><li>`I` - American Indian/Alaskan Native</li><li>`J` - Japanese</li><li>`K` - Korean</li><li>`L` - Laotian</li><li>`O` - Other</li><li>`P` - Pacific Islander</li><li>`S` - Samoan</li><li>`U` - Hawaiian</li><li>`V` - Vietnamese</li><li>`W` - White</li><li>`X` - Unknown</li><li>`Z` - Asian Indian</li> |
| `'Weapon Desc'` | Description of the weapon used (if applicable). |
| `'Status Desc'` | Crime status. |
| `'LOCATION'` | Street address of the crime. |

In [136]:
# Re-run this cell
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
crimes = pd.read_csv("crimes.csv", parse_dates=["Date Rptd", "DATE OCC"], dtype={"TIME OCC": str})
crimes.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status Desc,LOCATION
0,220314085,2022-07-22,2020-05-12,1110,Southwest,THEFT OF IDENTITY,27,F,B,NaN,Invest Cont,2500 S SYCAMORE AV
1,222013040,2022-08-06,2020-06-04,1620,Olympic,THEFT OF IDENTITY,60,M,H,NaN,Invest Cont,3300 SAN MARINO ST
2,220614831,2022-08-18,2020-08-17,1200,Hollywood,THEFT OF IDENTITY,28,M,H,NaN,Invest Cont,1900 TRANSIENT
3,231207725,2023-02-27,2020-01-27,0635,77th Street,THEFT OF IDENTITY,37,M,H,NaN,Invest Cont,6200 4TH AV
4,220213256,2022-07-14,2020-07-14,0900,Rampart,THEFT OF IDENTITY,79,M,B,NaN,Invest Cont,1200 W 7TH ST


In [137]:
from datetime import datetime

time_freq = { k:0 for k in range(0, 24)}

for t in crimes['TIME OCC']:
    time_freq[datetime.strptime(t, "%H%M").hour] += 1

peak_h = [k for k,v in time_freq.items() if v == max(time_freq.values())]

assert(len(peak_h) == 1)
peak_crime_hour = peak_h[0]

In [138]:
converted_dict = crimes.groupby('AREA NAME')['TIME OCC'].value_counts().to_dict()

locs = {l:0 for l, t in converted_dict.keys()}
for (l, t), f in converted_dict.items():
     if datetime.strptime(t, "%H%M").hour >=10 or datetime.strptime(t, "%H%M").hour <=4:
            locs[l] += f

peak_l = [k for k,v in locs.items() if v == max(locs.values())]

peak_night_crime_location = peak_l[0]

In [139]:
by_age_counts = {
    "0-17" : (crimes['Vict Age'] <= 17).sum(),
    "18-25" : ((crimes['Vict Age'] >= 18) & (crimes['Vict Age'] <= 25)).sum(),
    "26-34" : ((crimes['Vict Age'] >= 26) & (crimes['Vict Age'] <= 34)).sum(),
    "35-44" : ((crimes['Vict Age'] >= 35) & (crimes['Vict Age'] <= 44)).sum(),
    "45-54" : ((crimes['Vict Age'] >= 45) & (crimes['Vict Age'] <= 54)).sum(),
    "55-64" : ((crimes['Vict Age'] >= 55) & (crimes['Vict Age'] <= 64)).sum(),
    "65+" : (crimes['Vict Age'] >= 65).sum(),}
victim_ages = pd.Series(by_age_counts)